In [1]:
!pip install openai --upgrade

   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.9 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.9 MB ? eta -:--:--
   ---------- ----------------------------- 0.5/1.9 MB 670.4 kB/s eta 0:00:03
   ---------- ----------------------------- 0.5/1.9 MB 670.4 kB/s eta 0:00:03
   ---------------- ----------------------- 0.8/1.9 MB 713.3 kB/s eta 0:00:02
   ---------------- ----------------------- 0.8/1.9 MB 713.3 kB/s eta 0:00:02
   --------------------- ------------------ 1.0/1.9 MB 671.0 kB/s eta 0:00:02
   --------------------------- ------------ 1.3/1.9 MB 713.8 kB/s eta 0:00:01
   --------------------------- ------------ 1.3/1.9 MB 713.8 kB/s eta 0:00:01
   -------------------------------- ------- 1.6/1.9 MB 749.0 kB/s eta 0:00:01
   ----------------------------

In [2]:
key = ''


In [3]:
import os
import openai
from openai import OpenAI

In [4]:
openai.api_key=key
client=OpenAI(api_key=key)

In [5]:
def generate_gt(cve_description):
    """Generates the CWE mapping for a given CVE description using GPT-3.5."""
    prompt = f"""
    Analyze the following CVE description and map it to the appropriate CWE. 
    Provide a brief justification for your choice. 
    Ensure the last line of your response contains only the CWE ID.
    CVE Description: {cve_description}
    """

    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are an expert in cybersecurity and CWE mappings."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=150
        )

        if response.choices and hasattr(response.choices[0].message, 'content'):
            content = response.choices[0].message.content.strip()
            result = content.split('\n')[-1]  # Get the last line
            print(f"Generated CWE ID: {result}")  # Debug print
            return result
        else:
            print("Error: Invalid response from OpenAI API")
            return None

    except openai.RateLimitError as e:
        print(f"Rate limit error: {e}")
        time.sleep(2 ** random.randint(1, 5))  # Exponential backoff
        return generate_gt(cve_description)  # Retry

    except Exception as e:
        print(f"Error: {e}")
        return None

In [19]:
import pandas as pd 

df = pd.read_csv('myRCM.tsv', sep='\t').sample(n=200)
df['gpt3.5'] = df['Description'].apply(lambda x: generate_gt(x))
df.to_csv('gpt3.5mydata.tsv', sep='\t', index=False)


Generated CWE ID: CWE-79
Generated CWE ID: - The CWE-434 relates to scenarios where an attacker can upload a file with dangerous content and subsequently exploit it to perform unauthorized actions
Generated CWE ID: CWE-79
Generated CWE ID: The CVE description mentions an "OS Command Injection Vulnerability" in Dell Unity's svc_topstats utility. OS Command Injection vulnerabilities occur when an application incorporates user-controllable data into a command that is sent to a system shell. In this case, an authenticated attacker could exploit this vulnerability to execute arbitrary commands with root privileges, potentially leading to the ability to overwrite arbitrary files on the file system. CWE-78 precisely covers the improper neutralization of special elements used in an OS command, which aligns with the nature
Generated CWE ID: CWE-22
Generated CWE ID: CWE-89
Generated CWE ID: CWE-400
Generated CWE ID: CWE-119
Generated CWE ID: CWE-285
Generated CWE ID: CWE-89: Improper Neutralizat

In [20]:
def compute_rcm_accuracy(fname, col):
    """Compute RCM accuracy by comparing predicted CWE with ground truth."""
    df = pd.read_csv(fname, sep='\t')
    correct = 0
    total = 0

    for idx, row in df.iterrows():
        pred = row[col].upper()
        # gt = row['GT']
        gt = row['CWE']


        # Validate if the prediction starts with 'CWE-'
        if pred.startswith('CWE-'):
            total += 1
            if pred == gt:
                correct += 1
        else:
            print(f'Invalid response at row {idx + 1}')

    # Calculate and return the accuracy
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy: {accuracy:.2f}%")
    return accuracy



In [22]:
# Compute the accuracy using the saved predictions
accuracy = compute_rcm_accuracy('gpt3.5mydata.tsv', 'gpt3.5')
print(f"Final RCM Accuracy: {accuracy:.2f}%")

Invalid response at row 2
Invalid response at row 4
Invalid response at row 30
Invalid response at row 35
Invalid response at row 40
Invalid response at row 43
Invalid response at row 58
Invalid response at row 62
Invalid response at row 83
Invalid response at row 93
Invalid response at row 101
Invalid response at row 121
Invalid response at row 122
Invalid response at row 137
Invalid response at row 144
Invalid response at row 147
Invalid response at row 155
Invalid response at row 159
Invalid response at row 170
Invalid response at row 181
Accuracy: 49.44%
Final RCM Accuracy: 49.44%


In [12]:
d=pd.read_csv('/kaggle/working/gpt3.5.tsv',sep='\t')
d

,CVE_ID,Description,CWE,gpt3.5
0,CVE-2024-1076,The SSL Zen WordPress plugin before 4.6.0 doe...,NaN,CWE-538
1,CVE-2024-1009,A vulnerability was found in SourceCodester Em...,CWE-89,CWE-89: Improper Neutralization of Special Ele...
2,CVE-2024-0443,A flaw was found in the blkgs destruction path...,CWE-668,CWE-399
3,CVE-2024-0611,The Master Slider – Responsive Touch Slider pl...,NaN,CWE-79
4,CVE-2024-0709,The Cryptocurrency Widgets – Price Ticker & Co...,CWE-89,CWE-89
...,...,...,...,...
195,CVE-2024-0553,A vulnerability was found in GnuTLS. The respo...,CWE-203,CWE-208
196,CVE-2024-1024,A vulnerability has been found in SourceCodest...,CWE-79,CWE-79
197,CVE-2024-0346,A vulnerability has been found in CodeAstro Ve...,CWE-79,CWE-79
198,CVE-2024-0561,The Ultimate Posts Widget WordPress plugin bef...,NaN,CWE-79: Improper Neutralization of Input Durin...
